In [ ]:
# dataset
# https://www.kaggle.com/datasets/mathurinache/math-dataset
# !kaggle datasets download -d mathurinache/math-dataset

In [1]:
!!kaggle datasets download -d mathurinache/math-dataset
!unzip math-dataset.zip
!pip install datasets
!pip install transformers

Streaming output truncated to the last 5000 lines.
  inflating: MATH/train/geometry/1085.json  
  inflating: MATH/train/geometry/1086.json  
  inflating: MATH/train/geometry/1089.json  
  inflating: MATH/train/geometry/1090.json  
  inflating: MATH/train/geometry/1091.json  
  inflating: MATH/train/geometry/1094.json  
  inflating: MATH/train/geometry/1095.json  
  inflating: MATH/train/geometry/1099.json  
  inflating: MATH/train/geometry/11.json  
  inflating: MATH/train/geometry/1100.json  
  inflating: MATH/train/geometry/1101.json  
  inflating: MATH/train/geometry/1102.json  
  inflating: MATH/train/geometry/1104.json  
  inflating: MATH/train/geometry/1105.json  
  inflating: MATH/train/geometry/1106.json  
  inflating: MATH/train/geometry/1109.json  
  inflating: MATH/train/geometry/111.json  
  inflating: MATH/train/geometry/1110.json  
  inflating: MATH/train/geometry/1111.json  
  inflating: MATH/train/geometry/1113.json  
  inflating: MATH/train/geometry/1116.json  
  infla

In [2]:
import pandas as pd
import torch
import torch.nn as nn
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LIMIT = 1000
CONTEXTWINDOW = 512

print(DEVICE)

cpu


In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [30]:
# Load model directly

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
EOS_TOKEN = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

# Resize the model's embedding layer to include the new token
model.resize_token_embeddings(len(tokenizer))

Embedding(128257, 2048)

In [24]:
def tokenize_function(examples):

   return tokenizer(examples['text'],
                    padding="max_length",
                    max_length=CONTEXTWINDOW,
                    truncation=True,
                    return_tensors='pt')

In [8]:
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

def generate_prompt(examples):
    problems  = examples["problem"]
    solutions = examples["solution"]
    texts = []
    for problem, solution in zip(problems, solutions):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!

        texts.append(
            alpaca_prompt.format(problem, solution) + EOS_TOKEN
        )
    return { "text" : texts, }

In [25]:
dataset = load_dataset("json", data_dir='./MATH')
dataset = dataset.map(generate_prompt, batched=True)
tokenized_dataset = dataset.map(tokenize_function, batched=True).with_format("torch")

Resolving data files:   0%|          | 0/7500 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/5000 [00:00<?, ?it/s]

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

In [31]:
# """ Sample """

# input_tokens   = tokenized_dataset["train"][0]["input_ids"].unsqueeze(0).to(DEVICE)
# attention_mask = tokenized_dataset["train"][0]["attention_mask"].unsqueeze(0).to(DEVICE)

# generation_output = model.generate(
#       input_ids=input_tokens,
#       max_new_tokens=1000,
#       attention_mask=attention_mask,
#       temperature=0.3,
#       eos_token_id=tokenizer.eos_token_id,
#     )

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
